In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import matplotlib.pyplot as plt

In [ ]:
device = 'cpu'#torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device:', device)

for i in per_class_info:
    df = pd.read_csv(i)
    name_split = i.split('.csv')[0].split('-')
    if len(name_split) == 2:
        _, data = name_split
        continue
        
    else:
        _, data, model = name_split
        if data == 'mnist': 
            model= 'SmallCNN'
    
    if ('cifar' in data) & (len(name_split) > 2):
        label_pth = os.path.join('./', 'convexhull-feature', data, '{}'.format(model), '{}-test_target.pt'.format(data))
    elif data != 'mnist':
        label_pth = os.path.join('./', 'convexhull', data,'test_pixel_target.pt'.format(data))
    else:
        label_pth = os.path.join('./', 'convexhull-feature', data,'mnist-test_target.pt'.format(data))
        
    test_label = torch.load(label_pth).to(device)
    (test_ids, test_cnts) = torch.unique(test_label, sorted=True, return_counts=True)

    df = df[df.columns[1:]]
    df = np.array(df)
    print(i, df.shape)

    exterior_ids = []
    for i in range(test_label.shape[0]):
        if np.all(df[:, i] > 1):
            exterior_ids.append(i)
    exterior_ids = np.array(exterior_ids)
    
    closure_ids = []
    for i in range(test_label.shape[0]):
        if np.any(df[:, i] <= 1):
            closure_ids.append(i)
    closure_ids = np.array(closure_ids)
    
    print(len(exterior_ids), len(closure_ids))
    if data == 'cifar100':
        num_class = 100
    else:
        num_class = 10

    uncertain_model = [u for u in uncertainty_csv if (data == u.split('-')[0]) and (model == u.split('-')[1])]
    assert len(uncertain_model) == 1
    dfu = pd.read_csv('./arxived_code/{}'.format(uncertain_model[0]))
    pred = np.array(dfu['pred_class'])
    pred_soft = np.array(dfu['softmax'])
    
    exterior_labels = test_label[closure_ids]
    exterior_df = df[:, closure_ids]
    exterior_pred = pred[closure_ids]
    exterior_softmax = pred_soft[closure_ids]

    nearby_hulls = []
    top5_hulls = []
    hull_pred = []
    top5_pred = []

    n = 5
    for i in closure_ids:
        la = test_label[i]
        pred = pred[i]
        prob = pred_soft[i]
        dist = df[:, i]
        
        
        if np.argsort(tu)[0] == la.item():
            nearby_hulls.append(la)
        if la.item() in np.argsort(tu)[:n] :
            top5_hulls.append(la)
            
        if np.argsort(tu)[0] == pr.item():
            hull_pred.append(pr)
        if pr.item() in np.argsort(tu)[:n] :
            top5_pred.append(pr)
            
    
    # print(len(exterior_labels), len(nearby_hulls))
    print('top-1 acc Label: {:.3f} '.format(100*len(nearby_hulls)/len(exterior_labels)))
    print('top-{} acc Label: {:.3f} '.format(n, 100*len(top5_hulls)/len(exterior_labels)))
    print('top-1 acc Pred: {:.3f} '.format(100*len(hull_pred)/len(exterior_labels)))
    print('top-{} acc Pred: {:.3f} '.format(n, 100*len(top5_pred)/len(exterior_labels)))
            